Train a neural network using tensorflow to correlate basic oxide perovskite properties to OER activities.

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.inspection import permutation_importance

2025-05-16 09:54:45.040894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747414485.702806 1766580 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747414485.922361 1766580 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747414486.510769 1766580 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747414486.510792 1766580 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747414486.510795 1766580 computation_placer.cc:177] computation placer alr

Load, normalize and split the data.

In [4]:
# Load data
data = pd.read_excel("data_gp.xlsx")

# Select input features and target
features = ['t', 'mu', 'RA', 'XA', 'XB', 'QA', 'Nd', 'mu/t']
target = 'VRHE'
X = data[features].values
y = data[target].values

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
X_train = X_scaled[:18, :]
y_train = y[:18]
X_val = X_scaled[18:, :]
y_val = y[18:]

In [7]:
X_train, y_train

(array([[-1.08223085,  1.62676953, -0.77345319,  0.86166497, -3.46038823,
          0.97887708, -1.88632043,  1.48595843],
        [-0.95938445,  1.18516573, -0.77345319,  0.86166497, -1.30350668,
          0.97887708,  0.04477912,  1.16041762],
        [-0.83653805,  0.79876242, -0.77345319,  0.86166497,  0.85337488,
          0.97887708,  1.97587866,  0.83487681],
        [-1.20507726,  2.01317285, -0.77345319,  0.86166497, -1.36342005,
          0.97887708,  0.68847897,  1.77894515],
        [-0.81196877,  0.74356194, -0.77345319,  0.86166497,  0.75751347,
          0.97887708,  1.71839873,  0.83487681],
        [-0.81196877,  0.68836147, -0.77345319,  0.86166497,  0.66165207,
          0.97887708,  1.46091879,  0.80232273],
        [-0.68912236,  0.35715862, -0.77345319,  0.86166497, -0.10523915,
          0.97887708, -0.59892073,  0.54189008],
        [-0.66455308,  0.35715862, -0.71123065,  1.0650686 , -0.10523915,
          0.97887708, -0.59892073,  0.509336  ],
        [-0.6154

In [8]:
X_val, y_val

(array([[ 0.66066848, -0.1314678 ,  0.84433277, -0.18925382, -1.30350668,
         -0.23810524, -0.59892073, -0.42753331],
        [ 1.39284044, -0.17235704,  1.81500435, -0.81980509, -0.4946761 ,
         -0.96829463, -0.98514064, -0.78150648],
        [ 1.2612462 , -1.17414342,  1.16788996, -0.39943758,  0.85337488,
         -0.4815017 ,  1.20343885, -1.26611789],
        [ 0.82764462, -1.9714836 ,  0.2221074 , -1.17237139,  0.76350481,
         -1.45508756,  0.36662904, -1.51830759],
        [ 2.21775967, -2.4825991 ,  1.80878209, -1.7825823 ,  0.85337488,
         -1.45508756,  0.68847897, -2.34237777]]),
 array([1.6725    , 1.58616667, 1.62258333, 1.61941667, 1.60741667]))

Build a 3-layer neural network and train with 500 epochs.

In [12]:
# Build neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, epochs=500, verbose=1)

/global/homes/y/yeming/anaconda3/envs/ML-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1747337219.874511  579901 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 24423 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:c3:00.0, compute capability: 8.0


Epoch 1/500


I0000 00:00:1747337222.967005 2163802 service.cc:152] XLA service 0x7f3fc0005210 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747337222.967550 2163802 service.cc:160]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2025-05-15 12:27:03.035042: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747337223.248770 2163802 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 3.0967 - mae: 1.7151
Epoch 2/500


I0000 00:00:1747337224.895697 2163802 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - loss: 2.7887 - mae: 1.6225
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - loss: 2.5020 - mae: 1.5306
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - loss: 2.2355 - mae: 1.4392
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - loss: 1.9911 - mae: 1.3490
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - loss: 1.7667 - mae: 1.2602
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 1.5616 - mae: 1.1725
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - loss: 1.3756 - mae: 1.0866
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 1.2058 - mae: 1.0092
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - loss: 1.0542 - mae: 0.9386
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - loss: 0.9200 - mae: 0.8688
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - loss: 0.8015 - mae: 0.8000
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - loss: 0.6981 - mae: 0.7318
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/s

The MAE for training set is 0.0163 eV after 500 epochs of training. Evaluate the model with cross-validation set.

In [13]:
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f"Validation MAE from model.evaluate: {val_mae:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 1.0482 - mae: 0.9162
Validation MAE from model.evaluate: 0.9162


In [14]:
y_pred = model.predict(X_val)
mae = mean_absolute_error(y_pred, y_val)
print(f"Train MAE from mean_absolute_error: {mae:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
Train MAE from mean_absolute_error: 0.9162


The MAE for val set is huge! Need to reduce overfitting. Try increasing regularization.

In [15]:
# Rebuild neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, epochs=500, verbose=1)

Epoch 1/500


/global/homes/y/yeming/anaconda3/envs/ML-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 3.6663 - mae: 1.6924
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - loss: 3.4714 - mae: 1.6338
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - loss: 3.2850 - mae: 1.5753
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - loss: 3.1054 - mae: 1.5164
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 2.9320 - mae: 1.4571
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - loss: 2.7670 - mae: 1.3978
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - loss: 2.6097 - mae: 1.3384
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - loss: 2.4608 - mae: 1.2792
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - loss: 2.3192 - mae: 1.2199
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step - loss: 2.1853 - mae: 1.1606
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - loss: 2.0583 - mae: 1.1011
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - loss: 1.9388 - mae: 1.0415
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step 

In [16]:
# Evaluate model
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f"Validation MAE from model.evaluate: {val_mae:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 515ms/step - loss: 0.2015 - mae: 0.2927
Validation MAE from model.evaluate: 0.2927


For the first model fitting, the MAE for traing and val set are 0.0163 and 0.9162 eV, respectively. For the second model fitting, the MAE for traing and val set are 0.0248 and 0.2927 eV, respevtively, the overfitting is largely reduced, but not enough. Keeping reducing overfitting with grater regularization, and simultaneously reduce avoidable bias with more epochs.

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.1), input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.1)),
    tf.keras.layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, epochs=500, verbose=1)

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 10.9895 - mae: 1.7225
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 10.6877 - mae: 1.6480
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 10.4020 - mae: 1.5737
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 10.1333 - mae: 1.5001
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 9.8814 - mae: 1.4275
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 9.6448 - mae: 1.3559
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 9.4207 - mae: 1.2846
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 9.2086 - mae: 1.2176
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 9.0105 - mae: 1.1633
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 8.8262 - mae: 1.1101
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 8.6533 - mae: 1.0573
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 8.4915 - mae: 1.0053
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/s

In [10]:
# Evaluate model
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f"Validation MAE from model.evaluate: {val_mae:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step - loss: 0.0207 - mae: 0.0684
Validation MAE from model.evaluate: 0.0684


For the first model fitting, MAE(training) = 0.0163 eV, MAE(val) = 0.9162 eV.
For the second model fitting, MAE(training) = 0.0248 eV, MAE(val) = 0.2927 eV.
For the third model fitting, MAE(training) = 0.0222 eV, MAE(val) = 0.0684 eV.

Obviously, overfitting is reduced by the regularization. However, there is still room for improvement of avoidable bias. Since the MAE for training has reached minimum at epoch number of ~400, a more complicated model architecture can be considered, such as adding one more layer.

In [43]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.05), input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.05)),
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.05)),
    tf.keras.layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, epochs=500, verbose=1)

Epoch 1/500


2025-05-16 12:38:38.121139: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_227', 4 bytes spill stores, 4 bytes spill loads



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 9.5756 - mae: 1.8021
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 9.1783 - mae: 1.7012
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 8.8352 - mae: 1.6102
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 8.5641 - mae: 1.5382
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 8.3286 - mae: 1.4736
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 8.1044 - mae: 1.4096
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 7.8984 - mae: 1.3486
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 7.7026 - mae: 1.2880
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 7.5132 - mae: 1.2267
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 7.3288 - mae: 1.1639
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 7.1470 - mae: 1.0981
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 6.9678 - mae: 1.0287
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 6.79

In [44]:
# Evaluate model
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f"Validation MAE from model.evaluate: {val_mae:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 906ms/step - loss: 0.0605 - mae: 0.0247
Validation MAE from model.evaluate: 0.0247


The model is now better with MAE(train) = 0.0230 eV and MAE(val) = 0.0247 eV. 

Next step is to figure out the contribution of each feature to the fitted model’s performance, which can be measured by the permutation feature importance enabled by sci-kit learn.

In [45]:
from sklearn.inspection import permutation_importance

r = permutation_importance(model, X_val, y_val,
                           scoring='r2',
                           n_repeats=30,
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    print(f"{features[i]:<8}"
            f"{r.importances_mean[i]:.3f}"
            f" +/- {r.importances_std[i]:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━

In [46]:
r.importances.shape

(8, 30)

In [47]:
for i, feature in enumerate(features):
    print(f"\n{feature}:")
    print(r.importances[i])


t:
[ 0.14944468  0.15156026  0.06632298  0.09424647  0.0688664   0.
  0.15156026 -0.0846148   0.23330205  0.03142966  0.02530394  0.23330205
  0.02530394 -0.06042542  0.08901926  0.1028882   0.23577737  0.15156026
 -0.02361887  0.0953258   0.1836025   0.08604135  0.02530394 -0.01100374
  0.07181738  0.01457772  0.0461874   0.07245932  0.0084834   0.1500576 ]

mu:
[ 0.01629684  0.60719281 -0.00203187  0.45324559 -0.22108284  0.
  0.60719281  0.09752353  0.31513126 -0.22151202  0.15500291  0.31513126
  0.15500291  0.06655225  0.08284909  0.45387291  0.34925396  0.60719281
 -0.08340217  0.561897    0.19531409 -0.079665    0.15500291  0.11428929
 -0.06912086  0.00668828 -0.30824449  0.23708377  0.10416021  0.16061419]

RA:
[ 0.2747212   0.13634545  0.08218197 -0.10991938  0.22939051  0.
  0.13634545 -0.20123653  0.28436751  0.21568435  0.0648766   0.28436751
  0.0648766  -0.08281564  0.19190556 -0.10024465  0.33277654  0.13634545
  0.06542157 -0.13573421  0.24263766  0.15840617  0.0648766

In [48]:
features = ['t', 'mu', 'RA', 'XA', 'XB', 'QA', 'Nd']
target = 'VRHE'
X = data[features].values
y = data[target].values

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train = X_scaled[:18, :]
y_train = y[:18]
X_val = X_scaled[18:, :]
y_val = y[18:]

In [49]:
X_train, y_train

(array([[-1.08223085,  1.62676953, -0.77345319,  0.86166497, -3.46038823,
          0.97887708, -1.88632043],
        [-0.95938445,  1.18516573, -0.77345319,  0.86166497, -1.30350668,
          0.97887708,  0.04477912],
        [-0.83653805,  0.79876242, -0.77345319,  0.86166497,  0.85337488,
          0.97887708,  1.97587866],
        [-1.20507726,  2.01317285, -0.77345319,  0.86166497, -1.36342005,
          0.97887708,  0.68847897],
        [-0.81196877,  0.74356194, -0.77345319,  0.86166497,  0.75751347,
          0.97887708,  1.71839873],
        [-0.81196877,  0.68836147, -0.77345319,  0.86166497,  0.66165207,
          0.97887708,  1.46091879],
        [-0.68912236,  0.35715862, -0.77345319,  0.86166497, -0.10523915,
          0.97887708, -0.59892073],
        [-0.66455308,  0.35715862, -0.71123065,  1.0650686 , -0.10523915,
          0.97887708, -0.59892073],
        [-0.61541452,  0.35715862, -0.64900811,  1.26847224, -0.10523915,
          0.97887708, -0.59892073],
        [-

In [50]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.05), input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.05)),
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.05)),
    tf.keras.layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, epochs=500, verbose=1)

Epoch 1/500


/global/homes/y/yeming/anaconda3/envs/ML-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 9.2611 - mae: 1.7287
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 9.0189 - mae: 1.6724
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 8.8088 - mae: 1.6242
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 8.6160 - mae: 1.5798
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 8.4307 - mae: 1.5361
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 8.2494 - mae: 1.4916
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 8.0710 - mae: 1.4461
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 7.8964 - mae: 1.3998
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 7.7253 - mae: 1.3523
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 7.5549 - mae: 1.3026
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 7.3900 - mae: 1.2527
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 7.2292 - mae: 1.2020
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 7.07

In [51]:
# Evaluate model
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f"Validation MAE from model.evaluate: {val_mae:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step - loss: 0.0559 - mae: 0.0169
Validation MAE from model.evaluate: 0.0169


In [52]:
r = permutation_importance(model, X_val, y_val,
                           scoring='r2',
                           n_repeats=30,
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    print(f"{features[i]:<8}"
            f"{r.importances_mean[i]:.3f}"
            f" +/- {r.importances_std[i]:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━